# TD3 1/2 : prise en compte naïve de l'ordre des mots

In [3]:
import os

with open(os.path.join("SMSSpamCollection.txt")) as f:
    lines = [line.strip().split("\t") for line in f.readlines()]

text = [x[1] for x in lines]
y = [int(x[0] == "spam") for x in lines]

from tensorflow.keras.preprocessing.text import one_hot

# note that the index construction is based on hashing so that a small size will inscrease the risk of collision
vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in text]

#pad documents to a max length of 10 words

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(padded_docs, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)

On est obligé de calculer le OHE pour la description des données en entrée

In [3]:
train_X[0:5]

array([[1596, 4058, 3065, 7112, 4214, 4214, 4639, 3410, 6714, 4258],
       [7227, 9039, 1760, 9163, 2287, 9223, 8695, 3970, 7491, 7829],
       [7451, 1148,  534, 3696, 9503, 9670, 3694, 6567, 5384, 6572],
       [6836, 3675, 6420, 9137, 8978, 8888,    0,    0,    0,    0],
       [9028, 9116, 4488, 9889,   67, 9915, 6636, 2277, 9432, 7019]],
      dtype=int32)

In [5]:
from tensorflow.keras.utils import to_categorical

train_X_binary = to_categorical(train_X, num_classes=vocab_size)
test_X_binary = to_categorical(test_X, num_classes=vocab_size)

On concatène "à la main" les vecteurs pour chacun des mots qui constituent le document (opération "flatten" qu'on automatisera par la suite).

In [11]:
import numpy as np

nb_ex_train = train_X.shape[0]
new_train_X = []
for i in range(nb_ex_train):
    d = train_X_binary[i,:,:]
    d_r = d.reshape((max_length*vocab_size))
    new_train_X.append(d_r)
new_train_X = np.asarray(new_train_X)

nb_ex_test = test_X.shape[0]
new_test_X = []
for i in range(nb_ex_test):
    d = test_X_binary[i,:,:]
    d_r = d.reshape((max_length*vocab_size))
    new_test_X.append(d_r)
new_test_X = np.asarray(new_test_X)

Le modèle est un MLP qui comprend des poids associés à la position des mots dans le document (token).

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def seq_model():
    model = Sequential()
    model.add(Dense(8, activation='relu', input_dim=max_length*vocab_size))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

sm = seq_model()

# summarize the model
print(sm.summary())

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 800008    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 800,017
Trainable params: 800,017
Non-trainable params: 0
_________________________________________________________________
None


2021-12-07 10:34:30.666244: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 10:34:30.666530: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
train_y = np.array(train_y)
test_y = np.array(test_y)

In [14]:
sm.fit(new_train_X, train_y, epochs=10, verbose=1)

2021-12-07 10:34:41.054863: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 10:34:41.055080: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-07 10:34:41.199611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
122/122 [==============================] - 11s 85ms/step - loss: 0.5023 - acc: 0.8726
Epoch 2/10
122/122 [==============================] - 10s 83ms/step - loss: 0.2427 - acc: 0.9254
Epoch 3/10
122/122 [==============================] - 10s 83ms/step - loss: 0.1330 - acc: 0.9772
Epoch 4/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0778 - acc: 0.9895
Epoch 5/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0495 - acc: 0.9944
Epoch 6/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0336 - acc: 0.9959
Epoch 7/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0239 - acc: 0.9969
Epoch 8/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0176 - acc: 0.9979
Epoch 9/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0134 - acc: 0.9985
Epoch 10/10
122/122 [==============================] - 10s 83ms/step - loss: 0.0104 - acc: 0.9987


In [15]:
loss, accuracy = sm.evaluate(new_test_X, test_y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

2021-12-07 10:36:22.906864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Accuracy: 97.429770
